## Hamming codes

Hamming นิยามโค้ดสำหรับสร้างรหัสของข้อความขนาด 4 บิต เป็นรหัส codeword 7 บิต ที่สามารถตรวจสอบว่ารหัสมีความผิดพลาดและแก้ไขได้ ด้วยเมตริกซ์ $G$ ขนาด $7\times 4$ ดังด้านล่าง

In [38]:
G = [[1,0,0,0],
     [0,1,0,0],
     [0,0,1,0],
     [0,0,0,1],
     [0,1,1,1],
     [1,0,1,1],
     [1,1,0,1]
    ]
print(G)

[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 0, 1]]


สมมติว่าเรามีข้อความขนาด 4 บิต $x=[1,0,1,0]$ เราจะสร้างเวกเตอร์ได้ดังนี้

In [2]:
x = [1,0,1,0]
print(x)

[1, 0, 1, 0]


### คูณเมตริกซ์ด้วยเวกเตอร์

การทำงานหลักของเราคือการคูณเมตริกซ์ด้วยเวกเตอร์  ให้ปรับฟังก์ชัน `mvmult` จากคาบก่อนให้สามารถทำงานได้ในกรณีที่เมตริกซ์มีขนาดไม่จำเป็นต้องเป็น 2 แต่จำนวนคอลัมน์ของเมตริกซ์เท่ากับขนาดของเวกเตอร์ (นั่นคือคูณกันได้)

ในการเขียนให้เขียนฟังก์ชัน `vdot` ที่ทำการหา dot product ก่อน  อย่าลืมว่าเราทำงานใน $GF(2)$ ดังนั้นผลรวมอย่าลืม mod 2

*hint:* หาความยาวเวกเตอร์ด้วยฟังก์ชัน `len` อาจจะทดลองใช้การเขียนแบบ list comprehension (แต่ไม่จำเป็น)

In [11]:
def vdot(x1,x2):
    return sum([x1[i]*x2[i] for i in range(len(x1))])%2

ทดสอบด้วยโค้ดด้านล่าง ผลลัพธ์ควรเป็นดังนี้

    [1, 0, 1, 1, 1, 1, 0]

In [12]:
print([vdot([1,0,1],[0,1,1]),
       vdot([1,0,1,1],[0,1,1,1]),
       vdot([1,0,1,1,1],[0,1,1,1,1]),
       vdot([1,0,1,1,1,0],[0,1,1,1,1,0]),
       vdot([1,0,1,1,1,0],[0,1,1,1,1,1]),
       vdot([1,0,1,1,1,1],[0,1,1,1,1,0]),
       vdot([1,0,1,1,1,1],[0,1,1,1,1,1])])

[1, 0, 1, 1, 1, 1, 0]


ใช้ `vdot` ในการเขียน `mvmult`

In [14]:
def mvmult(M,x):
    ans = []
    for i in range(len(M)):
        ans.append(vdot(M[i],x))
    return ans

ทดสอบกับโค้ดด้านล่าง ผลลัพธ์ควรเป็นดังนี้

```
[1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 1, 0, 1, 0]
[1, 0, 0, 1, 1, 0, 0]
```

In [15]:
print(mvmult(G,[1,1,1,1]))
print(mvmult(G,[0,1,0,1]))
print(mvmult(G,[1,0,0,1]))

[1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 1, 0, 1, 0]
[1, 0, 0, 1, 1, 0, 0]


### Codeword

**codeword** คือข้อความที่ผ่านการเข้ารหัส (encode) แล้ว

codeword สำหรับข้อความ $x$ จะนิยามเป็น $Gx$ ซึ่งจะมีความยาว 7 บิต  ซึ่งเราคำนวณได้ดังด้านล่าง

In [16]:
# ฟังก์ชัน encode คูณเมตริกซ์ G เข้ากับ x บน GF(2)
def encode(G,x):
    return mvmult(G,x)

In [17]:
print(encode(G,[1,0,1,0]))

[1, 0, 1, 0, 1, 0, 1]


### ระยะระหว่าง codeword
รหัสจะมีคุณภาพที่ดีถ้า codeword ต่าง ๆ มีความแตกต่างกัน (เพราะว่าถ้ามีการเปลี่ยนแปลงแค่บางบิต ก็จะสามารถตรวจสอบได้)  ให้เขียนฟังก์ชัน `distance` ด้านล่างในการคำนวณจำนวนบิตที่แตกต่างกันของสอง codeword x กับ y

*หมายเหตุ* ระยะทางดังกล่าว เรียกว่า **Hamming distance**

*hint* มีฟังก์ชัน `abs` สำหรับหาค่าสัมบูรณ์

In [40]:
def distance(x,y):
    #print(x,y)
    return sum([(x[i]+y[i])%2 for i in range(len(x))])

In [24]:
print(distance(encode(G,[1,0,0,1]), encode(G,[1,1,1,1])))

4


ระยะระหว่าง codeword สองรหัสที่ใกล้กันที่สุดจะบอกความสามารถในการตรวจสอบความผิดพลาด (error detection) และการแก้ไขความผิดพลาด (error correction)

สำหรับ Hamming code ระยะใกล้สุดคือ 3

ให้หา message `x1` และ `x2` ที่มีระยะของ codeword เท่ากับ 3  (ให้ลองปรับ x1, x2 ดูด้านล่าง)

In [41]:
x1 = [1,1,0,0]
x2 = [1,1,1,1]
print(distance(encode(G,x1), encode(G,x2)))

3


**>>> คำถาม 1**:

จริง ๆ แล้วเราจะสามารถคู่ของ codeword ที่ใกล้ที่สุด โดยที่ให้ x1=[0,0,0,0] ได้เสมอ ทำไม?  ให้อธิบายเหตุผลคร่าว ๆ

สามารถนำ x1 ไปลบทั้ง x1 และ x2 โดยยังรักษา Hamming distance ไว้ได้

### Linear function

จากที่เราได้พิจารณาในห้องเรียน การคูณด้วยเมตริกซ์เป็น linear function นั่นคือถ้าเรานิยามให้ $f(x)=Mx$ เราจะได้ว่า $f$ เป็น linear function

ในกรณีนี้ `encode` (ขอเขียนย่อเป็น $e$) ที่นิยามด้วย $e(x) = Gx$ เป็นฟังก์ชันจาก $GF(2)^4$ ไปยัง $GF(2)^7$  หรือเขียนเป็น $e:GF(2)^4\rightarrow GF(2)^7$

**>>> คำถาม 2**:
1. $\mathsf{Ker}\; e$ คืออะไร? [0,0,0,0]
2. $\dim\; \mathsf{Ker}\; e$ เท่ากับเท่าใด 0
3. $\dim\; \mathsf{Im}\; e$ เท่ากับเท่าใด 4
4. $\dim\; GF(2)^4$ เท่ากับเท่าใด 4

### เกี่ยวกับ code

เราจะเรียกเซตของ codeword ทั้งหมดว่า code $C$ (หรือในอีกมุมหนึ่งอาจจะเห็นได้ว่า $C=\mathsf{Im}\; e$ นั่นเอง)

จากคำถามข้างต้น สังเกตว่า $C$ เป็นสตริง 7 บิต ดังนั้น $C\subset GF(2)^7$   เราทราบว่า $|GF(2)^7|=2^7$ แต่ขนาดของ $C$ เท่ากับเท่าใด?

**>>> คำถาม 3:** $|C|$ เท่ากับเท่าใด (พยายามใช้ความจริงเกี่ยวกับ $\mathsf{Ker}\; e$ ในการตอบ)

### Check matrix

Hamming นิยาม **check matrix** $H$ ดังนี้

In [44]:
H = [[0,0,0,1,1,1,1],
     [0,1,1,0,0,1,1],
     [1,0,1,0,1,0,1]]

คุณสมบัติหนึ่งของ check matrix คือ สำหรับทุก ๆ codeword $c\in C$, $Hc=0$  ให้เขียนเมทอดสำหรับแปลงเลขจำนวนเต็มเป็นเวกเตอร์ขนาด 4 บิตเพื่อใช้ทดลองด้านล่าง

In [59]:
#
# งานของคุณ
#
# แปลงจำนวนเต็ม i เป็นเวกเตอร์ของบิตความยาว n บิต
# เช่น bitvect(4,2) => [0,0,1,0]
#   bitvect(4,5) => [0,1,0,1]
#   bitvect(4,9) => [1,0,0,1]
def bitvect(n,i):
    return [(i//2**j)%2 for j in range(n)][::-1]

เมื่อเขียน bitvect เสร็จให้รันด้านล่าง  ผลลัพธ์ควรเป็น [0,0,0] ในทุกแถว

In [61]:
for i in range(16):
    print(mvmult(H, encode(G,bitvect(4,i))))

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]


ถ้าเรานิยามฟังก์ชัน $h(x): GF(2)^7\rightarrow GF(2)^3$ ดังนี้ $h(x)=Hx$  เราจะได้ว่า $h$ map ทุก ๆ codeword ไปยัง 0  ดังนั้นจากการทดลองข้างต้น เราทราบว่า $C\subset \mathsf{Ker}\; h$  

ยิ่งไปกว่านั้น อีกคุณสมบัติที่สำคัญก็คือ $C$ ไม่ใช่เป็น subset ของ $\mathsf{Ker}\; h$  แต่ $C=\mathsf{Ker}\; h$ (ขอละไม่พิสูจน์)

**ชวนคิด:** การที่ $C=\mathsf{Ker}\; h$ มีผลอย่างไรต่อการตรวจสอบความถูกต้องของข้อมูล

ด้านล่างให้ทดลองสร้าง codeword จากนั้นลองแก้บางบิตของ codeword ให้เป็นเวกเตอร์ $y$ แล้วหา $h(y)$

In [ ]:
# message ตั้งต้น
mx = [1,0,1,0] 
x = encode(G,mx)
print("x =", x)

In [ ]:
# look at x below and write y to be that codeword with 1 bit change
y = [1, 0, 0, 0, 1, 0, 1]
print("y =",y)
print("sindrome =", mvmult(H,y))

**>>> คำถาม 4:** 

1. ทดลองเปลี่ยนบิตที่ผิดไปหลาย ๆ ที่ สังเกตเห็นลักษณะพิเศษของผลลัพธ์ $h(y)$ หรือไม่?
2. ถ้าต้องการแก้ไข x ให้ผิดพลาดมากกว่า 1 ตำแหน่ง ต้องแก้อย่างน้อยกี่ตำแหน่งจึงจะทำให้ $h(y)=0$